In [1]:
import os
import requests
import zipfile

# Create the directory if it doesn't exist
raw_dir = "../data/raw"
if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)

# Download data as zip
url = "https://archive.ics.uci.edu/static/public/45/heart+disease.zip"
response = requests.get(url)

# Save the zip file to the specified directory
zip_path = os.path.join(raw_dir, "heart+disease.zip")
with open(zip_path, 'wb') as f:
    f.write(response.content)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(raw_dir)

print("Data extracted successfully!")

Data extracted successfully!


In [2]:
import numpy as np
import pandas as pd
# read in data
colnames = [
    "age",       
    "sex",       
    "cp",        
    "trestbps",  
    "chol",      
    "fbs",       
    "restecg",   
    "thalach",   
    "exang",     
    "oldpeak",   
    "slope",     
    "ca",        
    "thal",      
    "num"  
]

heart_disease = pd.read_csv("../data/raw/processed.cleveland.data", names=colnames, header=None)
# replace missing values with nan for ease of computational handling
heart_disease.replace('?', np.nan, inplace=True)
heart_disease

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


In [3]:
from sklearn import set_config
from sklearn.model_selection import train_test_split

# scale and split into train & test
np.random.seed(522)
set_config(transform_output="pandas")

# create the split
heart_disease_train, heart_disease_test = train_test_split(
    heart_disease, train_size=0.70, stratify=heart_disease["num"]
)
# Create the directory if it doesn't exist
processed_dir = "../data/processed"
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
heart_disease_train.to_csv("../data/processed/heart_disease_train.csv")
heart_disease_test.to_csv("../data/processed/heart_disease_test.csv")

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer, make_column_selector
heart_disease_preprocessor = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include='number')),
    remainder='passthrough',
    verbose_feature_names_out=False
)

heart_disease_preprocessor.fit(heart_disease_train)
scaled_heart_disease_train = heart_disease_preprocessor.transform(heart_disease_train)
scaled_heart_disease_test = heart_disease_preprocessor.transform(heart_disease_test)

scaled_heart_disease_train.to_csv("../data/processed/scaled_heart_disease_train.csv")
scaled_heart_disease_test.to_csv("../data/processed/scaled_heart_disease_test.csv")